In [1]:
%%time
!pip install keras==2.15.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 25.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: keras
    Found existing installation: keras 3.3.3
    Uninstalling keras-3.3.3:
      Successfully uninstalled keras-3.3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
CPU times: user 175 ms, sys: 58.9 ms, total: 234 ms
Wall time: 15.5 s


In [2]:
%%time
import keras
keras.__version__

2024-06-07 03:33:37.502853: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-07 03:33:37.502955: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-07 03:33:37.641546: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


CPU times: user 6.76 s, sys: 1.16 s, total: 7.93 s
Wall time: 12 s


'2.15.0'

In [3]:
%%time
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sn
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import vgg16
from keras.src.layers.pooling.average_pooling2d import AvgPool2D
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dropout
from keras.models import Sequential
from keras.layers import Dense,Input,Flatten
from tensorflow.keras.utils import load_img,img_to_array
from sklearn.metrics import confusion_matrix
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

CPU times: user 597 ms, sys: 60.6 ms, total: 657 ms
Wall time: 794 ms


In [4]:
%%time
import os
print(os.listdir("/kaggle/input/food-101/food-101/food-101/images/"))

['macarons', 'french_toast', 'lobster_bisque', 'prime_rib', 'pork_chop', 'guacamole', 'baby_back_ribs', 'mussels', 'beef_carpaccio', 'poutine', 'hot_and_sour_soup', 'seaweed_salad', 'foie_gras', 'dumplings', 'peking_duck', 'takoyaki', 'bibimbap', 'falafel', 'pulled_pork_sandwich', 'lobster_roll_sandwich', 'carrot_cake', 'beet_salad', 'panna_cotta', 'donuts', 'red_velvet_cake', 'grilled_cheese_sandwich', 'cannoli', 'spring_rolls', 'shrimp_and_grits', 'clam_chowder', 'omelette', 'fried_calamari', 'caprese_salad', 'oysters', 'scallops', 'ramen', 'grilled_salmon', 'croque_madame', 'filet_mignon', 'hamburger', 'spaghetti_carbonara', 'miso_soup', 'bread_pudding', 'lasagna', 'crab_cakes', 'cheesecake', 'spaghetti_bolognese', 'cup_cakes', 'creme_brulee', 'waffles', 'fish_and_chips', 'paella', 'macaroni_and_cheese', 'chocolate_mousse', 'ravioli', 'chicken_curry', 'caesar_salad', 'nachos', 'tiramisu', 'frozen_yogurt', 'ice_cream', 'risotto', 'club_sandwich', 'strawberry_shortcake', 'steak', 'chu

In [5]:
%%time
values = ['macarons', 'french_toast', 'lobster_bisque', 'prime_rib', 'pork_chop', 'guacamole', 'baby_back_ribs', 'mussels', 'beef_carpaccio', 'poutine', 'hot_and_sour_soup', 'seaweed_salad', 'foie_gras', 'dumplings', 'peking_duck', 'takoyaki', 'bibimbap', 'falafel', 'pulled_pork_sandwich', 'lobster_roll_sandwich', 'carrot_cake', 'beet_salad', 'panna_cotta', 'donuts', 'red_velvet_cake', 'grilled_cheese_sandwich', 'cannoli', 'spring_rolls', 'shrimp_and_grits', 'clam_chowder', 'omelette', 'fried_calamari', 'caprese_salad', 'oysters', 'scallops', 'ramen', 'grilled_salmon', 'croque_madame', 'filet_mignon', 'hamburger', 'spaghetti_carbonara', 'miso_soup', 'bread_pudding', 'lasagna', 'crab_cakes', 'cheesecake', 'spaghetti_bolognese', 'cup_cakes', 'creme_brulee', 'waffles', 'fish_and_chips', 'paella', 'macaroni_and_cheese', 'chocolate_mousse', 'ravioli', 'chicken_curry', 'caesar_salad', 'nachos', 'tiramisu', 'frozen_yogurt', 'ice_cream', 'risotto', 'club_sandwich', 'strawberry_shortcake', 'steak', 'churros', 'garlic_bread', 'baklava', 'bruschetta', 'hummus', 'chicken_wings', 'greek_salad', 'tuna_tartare', 'chocolate_cake', 'gyoza', 'eggs_benedict', 'deviled_eggs', 'samosa', 'sushi', 'breakfast_burrito', 'ceviche', 'beef_tartare', 'apple_pie', '.DS_Store', 'huevos_rancheros', 'beignets', 'pizza', 'edamame', 'french_onion_soup', 'hot_dog', 'tacos', 'chicken_quesadilla', 'pho', 'gnocchi', 'pancakes', 'fried_rice', 'cheese_plate', 'onion_rings', 'escargots', 'sashimi', 'pad_thai', 'french_fries']
values.sort()
values = values[1:]
print(values)

['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare', 'beet_salad', 'beignets', 'bibimbap', 'bread_pudding', 'breakfast_burrito', 'bruschetta', 'caesar_salad', 'cannoli', 'caprese_salad', 'carrot_cake', 'ceviche', 'cheese_plate', 'cheesecake', 'chicken_curry', 'chicken_quesadilla', 'chicken_wings', 'chocolate_cake', 'chocolate_mousse', 'churros', 'clam_chowder', 'club_sandwich', 'crab_cakes', 'creme_brulee', 'croque_madame', 'cup_cakes', 'deviled_eggs', 'donuts', 'dumplings', 'edamame', 'eggs_benedict', 'escargots', 'falafel', 'filet_mignon', 'fish_and_chips', 'foie_gras', 'french_fries', 'french_onion_soup', 'french_toast', 'fried_calamari', 'fried_rice', 'frozen_yogurt', 'garlic_bread', 'gnocchi', 'greek_salad', 'grilled_cheese_sandwich', 'grilled_salmon', 'guacamole', 'gyoza', 'hamburger', 'hot_and_sour_soup', 'hot_dog', 'huevos_rancheros', 'hummus', 'ice_cream', 'lasagna', 'lobster_bisque', 'lobster_roll_sandwich', 'macaroni_and_cheese', 'macarons', 'miso_sou

In [6]:
print("First element:",values[0],"\nLast element:",values[-1])

First element: apple_pie 
Last element: waffles


In [7]:
%%time

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create ImageDataGenerator with data augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1,
    rotation_range=20,       # Randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2,   # Randomly translate images horizontally (fraction of total width)
    height_shift_range=0.2,  # Randomly translate images vertically (fraction of total height)
    shear_range=0.2,         # Shear intensity (shear angle in counter-clockwise direction in degrees)
    zoom_range=0.2,          # Randomly zoom images
    horizontal_flip=True,    # Randomly flip images horizontally
    fill_mode='nearest'      # Strategy used for filling in newly created pixels
)

# Data generator for training data with augmentation
train_data = train_datagen.flow_from_directory(
    '/kaggle/input/food-101/food-101/food-101/images/',
    target_size=(224,224),
    batch_size=100,
    class_mode='categorical',
    shuffle=True,
    subset='training'
)

# Data generator for validation data (no augmentation, only rescaling)
validation_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1
)

test_data = validation_datagen.flow_from_directory(
    '/kaggle/input/food-101/food-101/food-101/images/',
    target_size=(224,224),
    batch_size=100,
    class_mode='categorical',
    shuffle=False,
    subset='validation'
)

Found 90900 images belonging to 101 classes.
Found 10100 images belonging to 101 classes.
CPU times: user 3.66 s, sys: 1.51 s, total: 5.17 s
Wall time: 8.43 s


# ResNet 50 Architecture

In [10]:
%%time
from keras.applications.Resnet50 import Resnet50
from keras.applications.Resnet50 import preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.layers import Input
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler, ReduceLROnPlateau
from keras.optimizers import SGD
from keras.regularizers import l2
import keras.backend as K
import math

CPU times: user 66 µs, sys: 0 ns, total: 66 µs
Wall time: 72.5 µs


In [ ]:
%%time
base_model2 = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))

x = base_model2.output
x = AveragePooling2D(2,2)(x)
x = Dropout(0.6)(x)
x = Flatten()(x)
x = Dense(101, kernel_initializer='glorot_uniform', kernel_regularizer=l2(.001), activation='softmax')(x)
model2 = Model(inputs=base_model2.input, outputs=x)
model2.summary()

87910968/87910968 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 111, 111, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 111, 111, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     

In [12]:
%%time
opt = SGD(lr=.1, momentum=.9)
model2.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

CPU times: user 25.1 ms, sys: 1.18 ms, total: 26.2 ms
Wall time: 24.1 ms


In [13]:
%%time
from tensorflow.keras.callbacks import EarlyStopping
results2 = model2.fit(train_data, epochs=20, validation_data=test_data,
                  steps_per_epoch=len(train_data), validation_steps=len(test_data),
                  callbacks = EarlyStopping(patience=7, monitor='val_loss',restore_best_weights=True)
                  )

Epoch 1/20


2024-06-07 03:35:10.810103: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
I0000 00:00:1717731319.262480     156 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


909/909 [==============================] - 1700s 2s/step - loss: 2.9691 - accuracy: 0.3513 - val_loss: 2.5404 - val_accuracy: 0.4342
Epoch 2/20
909/909 [==============================] - 1279s 1s/step - loss: 1.7009 - accuracy: 0.6136 - val_loss: 2.1513 - val_accuracy: 0.5178
Epoch 3/20
909/909 [==============================] - 1283s 1s/step - loss: 1.4094 - accuracy: 0.6805 - val_loss: 1.7704 - val_accuracy: 0.6014
Epoch 4/20
909/909 [==============================] - 1253s 1s/step - loss: 1.2447 - accuracy: 0.7160 - val_loss: 2.1288 - val_accuracy: 0.5273
Epoch 5/20
909/909 [==============================] - 1229s 1s/step - loss: 1.1170 - accuracy: 0.7442 - val_loss: 1.9395 - val_accuracy: 0.5872
Epoch 6/20
909/909 [==============================] - 1239s 1s/step - loss: 1.0303 - accuracy: 0.7624 - val_loss: 1.5220 - val_accuracy: 0.6589
Epoch 7/20
909/909 [==============================] - 1277s 1s/step - loss: 0.9536 - accuracy: 0.7809 - val_loss: 1.7236 - val_accuracy: 0.6231
Epo

In [14]:
loss, acc = model2.evaluate(test_data)

print("Test Accuracy:",round(acc*100,2),"%","\nTest Loss:",loss)

101/101 [==============================] - 44s 439ms/step - loss: 1.2093 - accuracy: 0.7312
Test Accuracy: 73.12 % 
Test Loss: 1.209320306777954
